In [89]:
import pandas as pd
import numpy as np
import pyodbc
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report
import warnings
from sklearn.exceptions import UndefinedMetricWarning


In [90]:
server = '192.168.1.212'
database = 'master'
username = 'test'
password = 'tester2024'

mssql_conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};'

In [93]:
msql_query = f"""
        select *
        from ProjectNew..FullCompanyInfo
        """
print(msql_query)


        select *
        from ProjectNew..FullCompanyInfo
        


In [94]:
try:
    mssql_conn = pyodbc.connect(mssql_conn_str)
    print("Kết nối cơ sở dữ liệu thành công")
except pyodbc.Error as e:
    print(f"Lỗi khi kết nối cơ sở dữ liệu: {e}")

data = pd.read_sql_query(msql_query, mssql_conn)

mssql_conn.close()

Kết nối cơ sở dữ liệu thành công


C:\Users\nnson\AppData\Local\Temp\ipykernel_18816\1879818091.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(msql_query, mssql_conn)


In [98]:
def processing_data(df1):
    df = df1.copy()

    # One-hot encoding cho cột CompanyType
    df = pd.get_dummies(df, columns=['CompanyType'], prefix='Type')
        
    if 'CompanyId' in df.columns:
        df.drop(columns=['CompanyId'], inplace=True)

    # for id in range(11, 25):
    #     mean_col_name = f'{id}_mean'
        
    #     # Tìm tất cả các cột tương ứng với id hiện tại
    #     cols_to_avg = [f'{year}_{id}' for year in range(2015, 2022)]
        
    #     # Tính giá trị trung bình và thêm vào cột mới
    #     df[mean_col_name] = df[cols_to_avg].mean(axis=1)

    # # Xóa các cột cũ
    # df.drop(columns=[f'{year}_{id}' for year in range(2015, 2022) for id in range(11, 25)], inplace=True)

    # Normalize các cột còn lại với giá trị từ 0 đến 1
    scaler = MinMaxScaler()
    df[df.columns] = scaler.fit_transform(df[df.columns])
    
    # Thay thế tất cả các giá trị NaN trong df thành 0
    df.fillna(0, inplace=True)
    
    return df

In [99]:
processed_data = processing_data(data)

In [100]:
processed_data

,CompanyAge,FDI,CapitalAmount,NumberOfLabors,Region,FS11,FS12,FS13,FS14,FS15,...,FS20,FS21,FS22,FS23,FS24,Status,Type_LLC1,Type_LLC2,Type_PE,Type_SC
0,0.176471,0.0,0.000292,0.000061,1.0,0.066675,0.000000,0.135581,0.023858,0.061939,...,0.071832,0.077137,0.033333,0.182198,0.217620,0.0,0.0,0.0,0.0,1.0
1,0.382353,0.0,0.000097,0.000076,0.0,0.066634,0.000000,0.135614,0.023858,0.061955,...,0.071798,0.077136,0.033332,0.182198,0.217620,1.0,0.0,0.0,0.0,1.0
2,0.147059,0.0,0.000015,0.000046,1.0,0.066630,0.000000,0.135581,0.023858,0.061939,...,0.071794,0.077136,0.033332,0.182198,0.217620,1.0,1.0,0.0,0.0,0.0
3,0.147059,0.0,0.000048,0.000015,1.0,0.066635,0.000000,0.135629,0.023858,0.061945,...,0.071799,0.077144,0.033334,0.182198,0.217620,1.0,1.0,0.0,0.0,0.0
4,0.088235,0.0,0.000002,0.000153,1.0,0.066627,0.000000,0.135581,0.023858,0.061939,...,0.071792,0.077136,0.033332,0.182198,0.217620,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235649,0.235294,0.0,0.000049,0.000153,1.0,0.066638,0.000000,0.000000,0.000000,0.000000,...,0.071801,0.000000,0.000000,0.182198,0.217620,1.0,1.0,0.0,0.0,0.0
235650,0.117647,0.0,0.000005,0.000015,1.0,0.066627,0.000000,0.135581,0.023858,0.061939,...,0.071792,0.077136,0.033332,0.182198,0.217620,1.0,0.0,1.0,0.0,0.0
235651,0.205882,0.0,0.000243,0.000153,1.0,0.066642,0.096372,0.135581,0.023917,0.061944,...,0.071841,0.077144,0.033339,0.182202,0.217624,1.0,1.0,0.0,0.0,0.0
235652,0.088235,0.0,0.000024,0.000015,1.0,0.066632,0.096354,0.135581,0.023858,0.061939,...,0.071796,0.077136,0.033332,0.182198,0.217620,1.0,1.0,0.0,0.0,0.0


In [102]:
def train_logistic_regression_kfold(processed_data, k=5):
    df = processed_data.copy()
    warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

    # Chia dữ liệu thành biến độc lập (X) và biến phụ thuộc (y)
    X = df.drop('Status', axis=1)
    y = df['Status']

    # Khởi tạo mô hình Logistic Regression
    model = LogisticRegression()

    # Khởi tạo KFold
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    accuracies = []
    reports = []

    fold = 1
    for train_index, test_index in kf.split(X):
        print(f"Training on fold {fold}...")
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Huấn luyện mô hình trên tập train
        model.fit(X_train, y_train)
        
        # Dự đoán trên tập test
        y_pred = model.predict(X_test)
        
        # Đánh giá mô hình
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        
        report = classification_report(y_test, y_pred, output_dict=True)
        reports.append(report)

        print(f"Accuracy for fold {fold}: {accuracy}")
        fold += 1

    # Tính toán và in ra báo cáo cuối cùng
    print("\nFinal Report:")
    print(f"Mean Accuracy: {np.mean(accuracies)}")
    print(f"Standard Deviation of Accuracy: {np.std(accuracies)}")
    
    # In ra classification report cho từng fold
    for i, report in enumerate(reports):
        print(f"\nClassification Report for fold {i+1}:")
        print(classification_report(y.iloc[test_index], y_pred))

In [103]:
train_logistic_regression_kfold(processed_data, 10)

Training on fold 1...
Accuracy for fold 1: 0.7954256131715183
Training on fold 2...
Accuracy for fold 2: 0.7940252906730034
Training on fold 3...
Accuracy for fold 3: 0.7948739709751337
Training on fold 4...
Accuracy for fold 4: 0.7954680471866248
Training on fold 5...
Accuracy for fold 5: 0.7959685974962869
Training on fold 6...
Accuracy for fold 6: 0.795374496074687
Training on fold 7...
Accuracy for fold 7: 0.7963505198387439
Training on fold 8...
Accuracy for fold 8: 0.7940165499681732
Training on fold 9...
Accuracy for fold 9: 0.7948652662847443
Training on fold 10...
Accuracy for fold 10: 0.7959261616804583

Final Report:
Mean Accuracy: 0.7952294513349374
Standard Deviation of Accuracy: 0.0007473360709186283

Classification Report for fold 1:
              precision    recall  f1-score   support

         0.0       0.38      0.00      0.01      4796
         1.0       0.80      1.00      0.89     18769

    accuracy                           0.80     23565
   macro avg       0.59

In [104]:
import xgboost as xgb

def train_xgboost_kfold(processed_data, k=5):
    df = processed_data.copy()
    warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
    
    # Chia dữ liệu thành biến độc lập (X) và biến phụ thuộc (y)
    X = df.drop('Status', axis=1)
    y = df['Status']

    # Khởi tạo KFold
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    accuracies = []
    reports = []

    fold = 1
    for train_index, test_index in kf.split(X):
        print(f"Training on fold {fold}...")
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Khởi tạo và huấn luyện mô hình XGBoost trên tập train
        model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
        model.fit(X_train, y_train)
        
        # Dự đoán trên tập test
        y_pred = model.predict(X_test)
        
        # Đánh giá mô hình
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        
        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
        reports.append(report)

        print(f"Accuracy for fold {fold}: {accuracy}")
        fold += 1

    # Tính toán và in ra báo cáo cuối cùng
    print("\nFinal Report:")
    print(f"Mean Accuracy: {np.mean(accuracies)}")
    print(f"Standard Deviation of Accuracy: {np.std(accuracies)}")
    
    # In ra classification report cho từng fold
    for i, report in enumerate(reports):
        print(f"\nClassification Report for fold {i+1}:")
        print(classification_report(y.iloc[test_index], y_pred, zero_division=0))


In [105]:
train_xgboost_kfold(processed_data, 10)

Training on fold 1...
Accuracy for fold 1: 0.7997963167274887
Training on fold 2...
Accuracy for fold 2: 0.8000084868030213
Training on fold 3...
Accuracy for fold 3: 0.8011542052108971
Training on fold 4...
Accuracy for fold 4: 0.8025120936943053
Training on fold 5...
Accuracy for fold 5: 0.8032675578187991
Training on fold 6...
Accuracy for fold 6: 0.8029705071079991
Training on fold 7...
Accuracy for fold 7: 0.8013579461065139
Training on fold 8...
Accuracy for fold 8: 0.8004243581582856
Training on fold 9...
Accuracy for fold 9: 0.8014852535539996
Training on fold 10...
Accuracy for fold 10: 0.8015276893698281

Final Report:
Mean Accuracy: 0.8014504414551137
Standard Deviation of Accuracy: 0.001127444727467978

Classification Report for fold 1:
              precision    recall  f1-score   support

         0.0       0.59      0.08      0.15      4796
         1.0       0.81      0.99      0.89     18769

    accuracy                           0.80     23565
   macro avg       0.70

Kết quả chạy mô hình XGBoost cho thấy độ chính xác (accuracy) tổng thể là 0.80, nhưng có sự chênh lệch lớn về hiệu suất giữa hai lớp. 

Status 0.0 có recall rất thấp (0.15), cho thấy mô hình gặp khó khăn trong việc dự đoán đúng các mẫu thuộc lớp này.

In [106]:
import lightgbm as lgb

def train_lightgbm_kfold(processed_data, k=5):
    df = processed_data.copy()
    warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
    
    # Chia dữ liệu thành biến độc lập (X) và biến phụ thuộc (y)
    X = df.drop('Status', axis=1)
    y = df['Status']

    # Khởi tạo KFold
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    accuracies = []
    reports = []

    fold = 1
    for train_index, test_index in kf.split(X):
        print(f"Training on fold {fold}...")
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Khởi tạo và huấn luyện mô hình LightGBM trên tập train
        model = lgb.LGBMClassifier()
        model.fit(X_train, y_train)
        
        # Dự đoán trên tập test
        y_pred = model.predict(X_test)
        
        # Đánh giá mô hình
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        
        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
        reports.append(report)

        print(f"Accuracy for fold {fold}: {accuracy}")
        fold += 1

    # Tính toán và in ra báo cáo cuối cùng
    print("\nFinal Report:")
    print(f"Mean Accuracy: {np.mean(accuracies)}")
    print(f"Standard Deviation of Accuracy: {np.std(accuracies)}")
    
    # In ra classification report cho từng fold
    for i, report in enumerate(reports):
        print(f"\nClassification Report for fold {i+1}:")
        print(classification_report(y.iloc[test_index], y_pred, zero_division=0))


In [107]:
train_lightgbm_kfold(processed_data, 10)

Training on fold 1...
[LightGBM] [Info] Number of positive: 168722, number of negative: 43366
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012842 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4126
[LightGBM] [Info] Number of data points in the train set: 212088, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.795528 -> initscore=1.358577
[LightGBM] [Info] Start training from score 1.358577
Accuracy for fold 1: 0.8003903929389798
Training on fold 2...
[LightGBM] [Info] Number of positive: 168779, number of negative: 43309
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003732 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4125
[LightGBM] [Info] Number of data points in the train set: 212088, number of used features:

Kết quả chạy mô hình LightGBM cho thấy độ chính xác (accuracy) tổng thể là 0.80, nhưng có sự chênh lệch lớn về hiệu suất giữa hai lớp. 

Status 0.0 có recall rất thấp (0.07), cho thấy mô hình gặp khó khăn trong việc dự đoán đúng các mẫu thuộc lớp này.

In [109]:
import catboost as cb

def train_catboost_kfold(processed_data, k=5):
    df = processed_data.copy()
    warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
    
    # Chia dữ liệu thành biến độc lập (X) và biến phụ thuộc (y)
    X = df.drop('Status', axis=1)
    y = df['Status']
    
    # Khởi tạo KFold
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    accuracies = []
    reports = []

    fold = 1
    for train_index, test_index in kf.split(X):
        print(f"Training on fold {fold}...")
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Khởi tạo và huấn luyện mô hình CatBoost trên tập train
        model = cb.CatBoostClassifier(verbose=0)
        model.fit(X_train, y_train)
        
        # Dự đoán trên tập test
        y_pred = model.predict(X_test)
        
        # Đánh giá mô hình
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        
        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
        reports.append(report)

        print(f"Accuracy for fold {fold}: {accuracy}")
        fold += 1

    # Tính toán và in ra báo cáo cuối cùng
    print("\nFinal Report:")
    print(f"Mean Accuracy: {np.mean(accuracies)}")
    print(f"Standard Deviation of Accuracy: {np.std(accuracies)}")
    
    # In ra classification report cho từng fold
    for i, report in enumerate(reports):
        print(f"\nClassification Report for fold {i+1}:")
        print(classification_report(y.iloc[test_index], y_pred, zero_division=0))


In [110]:
train_catboost_kfold(processed_data, 10)

Training on fold 1...
Accuracy for fold 1: 0.8006449970296189
Training on fold 2...
Accuracy for fold 2: 0.800814733090045
Training on fold 3...
Accuracy for fold 3: 0.8011966392260036
Training on fold 4...
Accuracy for fold 4: 0.8037851141475006
Training on fold 5...
Accuracy for fold 5: 0.8046679397411415
Training on fold 6...
Accuracy for fold 6: 0.8034373010821133
Training on fold 7...
Accuracy for fold 7: 0.8018671758964566
Training on fold 8...
Accuracy for fold 8: 0.8028431996605134
Training on fold 9...
Accuracy for fold 9: 0.8030978145554848
Training on fold 10...
Accuracy for fold 10: 0.8027583280288564

Final Report:
Mean Accuracy: 0.8025113242457735
Standard Deviation of Accuracy: 0.0012708662352027091

Classification Report for fold 1:
              precision    recall  f1-score   support

         0.0       0.61      0.09      0.15      4796
         1.0       0.81      0.99      0.89     18769

    accuracy                           0.80     23565
   macro avg       0.71

In [111]:
# train_hist_gradient_boosting_kfold
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier


def train_hist_gradient_boosting_kfold(processed_data, k=5):
    # Sao chép DataFrame và bỏ qua cảnh báo UndefinedMetricWarning
    df = processed_data.copy()
    warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
    
    # Chia dữ liệu thành biến độc lập (X) và biến phụ thuộc (y)
    X = df.drop('Status', axis=1)
    y = df['Status']

    # Khởi tạo KFold
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    accuracies = []
    reports = []

    fold = 1
    for train_index, test_index in kf.split(X):
        print(f"Training on fold {fold}...")
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Khởi tạo và huấn luyện mô hình HistGradientBoostingClassifier trên tập train
        model = HistGradientBoostingClassifier()
        model.fit(X_train, y_train)
        
        # Dự đoán trên tập test
        y_pred = model.predict(X_test)
        
        # Đánh giá mô hình
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        
        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
        reports.append(report)

        print(f"Accuracy for fold {fold}: {accuracy}")
        fold += 1

    # Tính toán và in ra báo cáo cuối cùng
    print("\nFinal Report:")
    print(f"Mean Accuracy: {np.mean(accuracies)}")
    print(f"Standard Deviation of Accuracy: {np.std(accuracies)}")
    
    # In ra classification report cho từng fold
    for i, report in enumerate(reports):
        print(f"\nClassification Report for fold {i+1}:")
        print(classification_report(y.iloc[test_index], y_pred, zero_division=0))


c:\Users\nnson\miniconda3\Lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:15: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [114]:
# train_random_forest_kfold
from sklearn.ensemble import RandomForestClassifier

def train_random_forest_kfold(processed_data, k=5):
    # Sao chép DataFrame và bỏ qua cảnh báo UndefinedMetricWarning
    df = processed_data.copy()
    warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
    
    # Chia dữ liệu thành biến độc lập (X) và biến phụ thuộc (y)
    X = df.drop('Status', axis=1)
    y = df['Status']

    # Khởi tạo KFold
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    accuracies = []
    reports = []

    fold = 1
    for train_index, test_index in kf.split(X):
        print(f"Training on fold {fold}...")
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Khởi tạo và huấn luyện mô hình RandomForestClassifier trên tập train
        model = RandomForestClassifier()
        model.fit(X_train, y_train)
        
        # Dự đoán trên tập test
        y_pred = model.predict(X_test)
        
        # Đánh giá mô hình
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        
        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
        reports.append(report)

        print(f"Accuracy for fold {fold}: {accuracy}")
        fold += 1

    # Tính toán và in ra báo cáo cuối cùng
    print("\nFinal Report:")
    print(f"Mean Accuracy: {np.mean(accuracies)}")
    print(f"Standard Deviation of Accuracy: {np.std(accuracies)}")
    
    # In ra classification report cho từng fold
    for i, report in enumerate(reports):
        print(f"\nClassification Report for fold {i+1}:")
        print(classification_report(y.iloc[test_index], y_pred, zero_division=0))


In [123]:
# train_random_forest_class_weight_kfold

# Imbalanced Learning Techniques - Class Weight Adjustment: Điều chỉnh trọng số lớp để mô hình tập trung hơn vào lớp thiểu số.

def train_random_forest_class_weight_kfold(processed_data, k=5):
    # Sao chép DataFrame và bỏ qua cảnh báo UndefinedMetricWarning
    df = processed_data.copy()
    warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
    
    # Chia dữ liệu thành biến độc lập (X) và biến phụ thuộc (y)
    X = df.drop('Status', axis=1)
    y = df['Status']

    # Khởi tạo KFold
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    accuracies = []
    reports = []

    fold = 1
    for train_index, test_index in kf.split(X):
        print(f"Training on fold {fold}...")
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Khởi tạo và huấn luyện mô hình RandomForestClassifier với class_weight='balanced'
        model = RandomForestClassifier(class_weight='balanced')
        model.fit(X_train, y_train)
        
        # Dự đoán trên tập test
        y_pred = model.predict(X_test)
        
        # Đánh giá mô hình
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        
        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
        reports.append(report)

        print(f"Accuracy for fold {fold}: {accuracy}")
        fold += 1

    # Tính toán và in ra báo cáo cuối cùng
    print("\nFinal Report:")
    print(f"Mean Accuracy: {np.mean(accuracies)}")
    print(f"Standard Deviation of Accuracy: {np.std(accuracies)}")
    
    # In ra classification report cho từng fold
    for i, report in enumerate(reports):
        print(f"\nClassification Report for fold {i+1}:")
        print(classification_report(y.iloc[test_index], y_pred, zero_division=0))


In [126]:
# train_random_forest_grid_search

# Hyperparameter Tuning - Grid Search

from sklearn.model_selection import KFold, GridSearchCV

def train_random_forest_grid_search(processed_data, k=5):
    # Sao chép DataFrame và bỏ qua cảnh báo UndefinedMetricWarning
    df = processed_data.copy()
    warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
    
    # Chia dữ liệu thành biến độc lập (X) và biến phụ thuộc (y)
    X = df.drop('Status', axis=1)
    y = df['Status']

    # Khởi tạo KFold
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    # Thiết lập các tham số cho Grid Search
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    # Khởi tạo Grid Search với RandomForestClassifier
    grid_search = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv=kf, scoring='accuracy', n_jobs=-1, verbose=2)

    # Huấn luyện Grid Search trên toàn bộ dữ liệu
    grid_search.fit(X, y)

    # Lấy mô hình tốt nhất từ Grid Search
    best_model = grid_search.best_estimator_

    # In ra các tham số tốt nhất
    print("Best parameters found: ", grid_search.best_params_)
    print("Best cross-validation accuracy: ", grid_search.best_score_)

    # Dự đoán và đánh giá mô hình tốt nhất trên toàn bộ tập dữ liệu
    y_pred = best_model.predict(X)
    print("Classification Report for best model:")
    print(classification_report(y, y_pred, zero_division=0))

In [127]:
# train_random_forest_random_search

# Hyperparameter Tuning - Random Search

from sklearn.model_selection import KFold, RandomizedSearchCV

def train_random_forest_random_search(processed_data, k=5):
    # Sao chép DataFrame và bỏ qua cảnh báo UndefinedMetricWarning
    df = processed_data.copy()
    warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
    
    # Chia dữ liệu thành biến độc lập (X) và biến phụ thuộc (y)
    X = df.drop('Status', axis=1)
    y = df['Status']

    # Khởi tạo KFold
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    # Thiết lập các tham số cho Random Search
    param_dist = {
        'n_estimators': randint(50, 200),
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': randint(2, 11),
        'min_samples_leaf': randint(1, 5)
    }

    # Khởi tạo Random Search với RandomForestClassifier
    random_search = RandomizedSearchCV(estimator=RandomForestClassifier(), param_distributions=param_dist, n_iter=100, cv=kf, scoring='accuracy', n_jobs=-1, verbose=2, random_state=42)

    # Huấn luyện Random Search trên toàn bộ dữ liệu
    random_search.fit(X, y)

    # Lấy mô hình tốt nhất từ Random Search
    best_model = random_search.best_estimator_

    # In ra các tham số tốt nhất
    print("Best parameters found: ", random_search.best_params_)
    print("Best cross-validation accuracy: ", random_search.best_score_)

    # Dự đoán và đánh giá mô hình tốt nhất trên toàn bộ tập dữ liệu
    y_pred = best_model.predict(X)
    print("Classification Report for best model:")
    print(classification_report(y, y_pred, zero_division=0))

# Giả sử processed_data là DataFrame của bạn
# processed_data = pd.read_csv('your_processed_data.csv')

# Gọi hàm với số lần fold là 5
# train_random_forest_random_search(processed_data, k=5)


In [115]:
# train_stacking_kfold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression

def train_stacking_kfold(processed_data, k=5):
    # Sao chép DataFrame và bỏ qua cảnh báo UndefinedMetricWarning
    df = processed_data.copy()
    warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
    
    # Chia dữ liệu thành biến độc lập (X) và biến phụ thuộc (y)
    X = df.drop('Status', axis=1)
    y = df['Status']

    # Khởi tạo KFold
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    accuracies = []
    reports = []

    fold = 1
    for train_index, test_index in kf.split(X):
        print(f"Training on fold {fold}...")
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Khởi tạo các mô hình cơ bản
        estimators = [
            ('rf', RandomForestClassifier()),
            ('gb', GradientBoostingClassifier())
        ]
        
        # Khởi tạo và huấn luyện mô hình StackingClassifier trên tập train
        model = StackingClassifier(
            estimators=estimators,
            final_estimator=LogisticRegression()
        )
        model.fit(X_train, y_train)
        
        # Dự đoán trên tập test
        y_pred = model.predict(X_test)
        
        # Đánh giá mô hình
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        
        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
        reports.append(report)

        print(f"Accuracy for fold {fold}: {accuracy}")
        fold += 1

    # Tính toán và in ra báo cáo cuối cùng
    print("\nFinal Report:")
    print(f"Mean Accuracy: {np.mean(accuracies)}")
    print(f"Standard Deviation of Accuracy: {np.std(accuracies)}")
    
    # In ra classification report cho từng fold
    for i, report in enumerate(reports):
        print(f"\nClassification Report for fold {i+1}:")
        print(classification_report(y.iloc[test_index], y_pred, zero_division=0))

# Giả sử processed_data là DataFrame của bạn
# processed_data = pd.read_csv('your_processed_data.csv')

# Gọi hàm với số lần fold là 5
# train_stacking_kfold(processed_data, k=5)


In [136]:
# train_deep_learning_kfold
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ProgbarLogger

def build_model(input_shape):
    model = Sequential()
    model.add(Dense(64, input_dim=input_shape, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def train_deep_learning_kfold(processed_data, k=5, epochs=50, batch_size=32):
    df = processed_data.copy()
    warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
    
    X = df.drop('Status', axis=1)
    y = df['Status']

    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    accuracies = []
    reports = []

    fold = 1
    for train_index, test_index in kf.split(X):
        print(f"Training on fold {fold}...")
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Chuẩn hóa dữ liệu
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        
        # Xây dựng mô hình
        model = build_model(X_train.shape[1])
        
        # Đào tạo mô hình với tpdm (ProgbarLogger)
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1, 
                  validation_data=(X_test, y_test), callbacks=[ProgbarLogger()])
        
        # Dự đoán trên tập test
        y_pred = (model.predict(X_test) > 0.5).astype("int32")
        
        # Đánh giá mô hình
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        
        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
        reports.append(report)

        print(f"Accuracy for fold {fold}: {accuracy}")
        fold += 1

    # Tính toán và in ra báo cáo cuối cùng
    print("\nFinal Report:")
    print(f"Mean Accuracy: {np.mean(accuracies)}")
    print(f"Standard Deviation of Accuracy: {np.std(accuracies)}")
    
    # In ra classification report cho từng fold
    for i, report in enumerate(reports):
        print(f"\nClassification Report for fold {i+1}:")
        print(classification_report(y.iloc[test_index], y_pred, zero_division=0))



In [120]:
# train_automl_kfold with tpot
from tpot import TPOTClassifier

def train_automl_kfold(processed_data, k=5, generations=50, population_size=50):
    df = processed_data.copy()
    warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
    
    X = df.drop('Status', axis=1)
    y = df['Status']

    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    accuracies = []
    reports = []

    fold = 1
    for train_index, test_index in kf.split(X):
        print(f"Training on fold {fold}...")
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Khởi tạo và huấn luyện mô hình TPOTClassifier trên tập train
        model = TPOTClassifier(generations=generations, population_size=population_size, verbosity=2, random_state=42)
        model.fit(X_train, y_train)
        
        # Dự đoán trên tập test
        y_pred = model.predict(X_test)
        
        # Đánh giá mô hình
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        
        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
        reports.append(report)

        print(f"Accuracy for fold {fold}: {accuracy}")
        fold += 1

    # Tính toán và in ra báo cáo cuối cùng
    print("\nFinal Report:")
    print(f"Mean Accuracy: {np.mean(accuracies)}")
    print(f"Standard Deviation of Accuracy: {np.std(accuracies)}")
    
    # In ra classification report cho từng fold
    for i, report in enumerate(reports):
        print(f"\nClassification Report for fold {i+1}:")
        print(classification_report(y.iloc[test_index], y_pred, zero_division=0))


In [124]:
# train_smote_kfold
from imblearn.over_sampling import SMOTE

def train_smote_kfold(processed_data, k=5):
    df = processed_data.copy()
    warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
    
    X = df.drop('Status', axis=1)
    y = df['Status']

    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    accuracies = []
    reports = []

    fold = 1
    for train_index, test_index in kf.split(X):
        print(f"Training on fold {fold}...")
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Áp dụng SMOTE cho tập train
        smote = SMOTE(random_state=42)
        X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
        
        # Khởi tạo và huấn luyện mô hình RandomForestClassifier trên tập train đã được resample
        model = RandomForestClassifier(random_state=42)
        model.fit(X_train_res, y_train_res)
        
        # Dự đoán trên tập test
        y_pred = model.predict(X_test)
        
        # Đánh giá mô hình
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        
        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
        reports.append(report)

        print(f"Accuracy for fold {fold}: {accuracy}")
        fold += 1

    # Tính toán và in ra báo cáo cuối cùng
    print("\nFinal Report:")
    print(f"Mean Accuracy: {np.mean(accuracies)}")
    print(f"Standard Deviation of Accuracy: {np.std(accuracies)}")
    
    # In ra classification report cho từng fold
    for i, report in enumerate(reports):
        print(f"\nClassification Report for fold {i+1}:")
        print(classification_report(y.iloc[test_index], y_pred, zero_division=0))

ImportError: cannot import name '_deprecate_Xt_in_inverse_transform' from 'sklearn.utils.deprecation' (c:\Users\nnson\miniconda3\Lib\site-packages\sklearn\utils\deprecation.py)

In [128]:
train_hist_gradient_boosting_kfold(processed_data, 10)

Training on fold 1...
Accuracy for fold 1: 0.7995841466519562
Training on fold 2...
Accuracy for fold 2: 0.8006025630145124
Training on fold 3...
Accuracy for fold 3: 0.800899601120258
Training on fold 4...
Accuracy for fold 4: 0.8022150555885598
Training on fold 5...
Accuracy for fold 5: 0.8030129429238277
Training on fold 6...
Accuracy for fold 6: 0.8019944833439423
Training on fold 7...
Accuracy for fold 7: 0.8028007638446849
Training on fold 8...
Accuracy for fold 8: 0.801060895395714
Training on fold 9...
Accuracy for fold 9: 0.7998302567366857
Training on fold 10...
Accuracy for fold 10: 0.8017398684489709

Final Report:
Mean Accuracy: 0.8013740577069111
Standard Deviation of Accuracy: 0.0011159276334650715

Classification Report for fold 1:
              precision    recall  f1-score   support

         0.0       0.62      0.07      0.12      4796
         1.0       0.81      0.99      0.89     18769

    accuracy                           0.80     23565
   macro avg       0.71 

In [129]:
train_random_forest_kfold(processed_data, 10)

Training on fold 1...
Accuracy for fold 1: 0.7945769328693881
Training on fold 2...
Accuracy for fold 2: 0.7942798947636426
Training on fold 3...
Accuracy for fold 3: 0.7941950267334296
Training on fold 4...
Accuracy for fold 4: 0.7950012730204532
Training on fold 5...
Accuracy for fold 5: 0.7957564184171441
Training on fold 6...
Accuracy for fold 6: 0.7966051347337153
Training on fold 7...
Accuracy for fold 7: 0.7957139826013155
Training on fold 8...
Accuracy for fold 8: 0.7949501379164015
Training on fold 9...
Accuracy for fold 9: 0.7963080840229153
Training on fold 10...
Accuracy for fold 10: 0.7952896244430299

Final Report:
Mean Accuracy: 0.7952676509521435
Standard Deviation of Accuracy: 0.0007800641290058936

Classification Report for fold 1:
              precision    recall  f1-score   support

         0.0       0.49      0.15      0.23      4796
         1.0       0.82      0.96      0.88     18769

    accuracy                           0.80     23565
   macro avg       0.6

In [130]:
train_random_forest_class_weight_kfold(processed_data, 10)

Training on fold 1...
Accuracy for fold 1: 0.7674615972163286
Training on fold 2...
Accuracy for fold 2: 0.7678435033522872
Training on fold 3...
Accuracy for fold 3: 0.762496817448867
Training on fold 4...
Accuracy for fold 4: 0.7661037087329203
Training on fold 5...
Accuracy for fold 5: 0.7672395501803522
Training on fold 6...
Accuracy for fold 6: 0.7691915977084659
Training on fold 7...
Accuracy for fold 7: 0.7662635264162954
Training on fold 8...
Accuracy for fold 8: 0.7648631444939529
Training on fold 9...
Accuracy for fold 9: 0.766390833863781
Training on fold 10...
Accuracy for fold 10: 0.768512624655209

Final Report:
Mean Accuracy: 0.7666366904068459
Standard Deviation of Accuracy: 0.001825233375266761

Classification Report for fold 1:
              precision    recall  f1-score   support

         0.0       0.41      0.29      0.34      4796
         1.0       0.83      0.89      0.86     18769

    accuracy                           0.77     23565
   macro avg       0.62   

In [131]:
train_random_forest_grid_search(processed_data, 10)

Fitting 10 folds for each of 108 candidates, totalling 1080 fits


ModuleNotFoundError: No module named 'joblib.externals.loky.backend.popen_loky_win32'

In [ ]:
train_random_forest_random_search(processed_data, 10)

In [132]:
train_stacking_kfold(processed_data, 10)

Training on fold 1...
Accuracy for fold 1: 0.8011542052108971
Training on fold 2...
Accuracy for fold 2: 0.7992446745311041
Training on fold 3...
Accuracy for fold 3: 0.7997114486972757
Training on fold 4...
Accuracy for fold 4: 0.8003055249087668
Training on fold 5...
Accuracy for fold 5: 0.8009335879482283
Training on fold 6...
Accuracy for fold 6: 0.8014003819223424
Training on fold 7...
Accuracy for fold 7: 0.8009760237640569
Training on fold 8...
Accuracy for fold 8: 0.8025885847655421
Training on fold 9...
Accuracy for fold 9: 0.8000424358158286
Training on fold 10...
Accuracy for fold 10: 0.8011882028431997

Final Report:
Mean Accuracy: 0.8007545070407242
Standard Deviation of Accuracy: 0.000910284629604158

Classification Report for fold 1:
              precision    recall  f1-score   support

         0.0       0.57      0.09      0.16      4796
         1.0       0.81      0.98      0.89     18769

    accuracy                           0.80     23565
   macro avg       0.69

In [137]:
train_deep_learning_kfold(processed_data, 10)

Training on fold 1...
Epoch 1/50


c:\Users\nnson\miniconda3\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


 700/6628 ━━━━━━━━━━━━━━━━━━━━ 4s 801us/step - accuracy: 0.7935 - loss: 0.5103

KeyboardInterrupt: 

In [ ]:
train_automl_kfold(processed_data, 10)